In [ ]:
!pip install -r requirements.txt

In [ ]:
import os

KAFKA_BOOTSTRAP_SERVER = os.environ.get('KAFKA_BOOTSTRAP_SERVER')
KAFKA_SECURITY_PROTOCOL = os.environ.get('KAFKA_SECURITY_PROTOCOL')
KAFKA_SASL_MECHANISM = os.environ.get('KAFKA_SASL_MECHANISM')
KAFKA_USERNAME = os.environ.get('KAFKA_USERNAME')
KAFKA_PASSWORD = os.environ.get('KAFKA_PASSWORD')

KAFKA_TOPIC_IMAGES = os.environ.get('KAFKA_TOPIC_IMAGES')
KAFKA_TOPIC_OBJECTS = os.environ.get('KAFKA_TOPIC_OBJECTS')
KAFKA_TOPIC_NOTEBOOKS = 'notebook-test'

KAFKA_CONSUMER_GROUP = 'notebook-save-files'

In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json

def create_consumer_save_files():
    consumer = KafkaConsumer(KAFKA_TOPIC_IMAGES,
                             group_id=KAFKA_CONSUMER_GROUP,
                             bootstrap_servers=[KAFKA_BOOTSTRAP_SERVER],
                             security_protocol=KAFKA_SECURITY_PROTOCOL,
                             sasl_mechanism=KAFKA_SASL_MECHANISM,
                             sasl_plain_username=KAFKA_USERNAME,
                             sasl_plain_password=KAFKA_PASSWORD,
                             auto_offset_reset='earliest',
                             api_version_auto_timeout_ms=30000,
                             request_timeout_ms=450000)

    print(f'Subscribed to "{KAFKA_BOOTSTRAP_SERVER}" consuming topic "{KAFKA_TOPIC_IMAGES}"...')

    try:
        for record in consumer:
            timestamp = record.timestamp
            filename = f'messages/{timestamp}.json'
            msg = record.value.decode('utf-8')
            topic = record.topic
            json_data = json.loads(msg)
            print(('Received the following message on the '
                   f'topic "{topic}": {msg}'))
            with open(filename, 'w') as outfile:
                print(f'Writing msg to "{filename}"')
                outfile.write(msg)

    finally:
        print("Closing consumer...")
        consumer.close()
    print("Kafka consumer stopped.")



In [ ]:

try:
    create_consumer_save_files()
except KeyboardInterrupt:
    print('Stopped')
